In [ ]:
import SimpleITK as sitk
import matplotlib.pyplot as plt
from scipy import ndimage

import numpy as np
import pandas as pd
import os
import json

import random
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers


In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)


In [ ]:
def read_nifti_file(file_path):
    """Read and load volume"""
    # Read file
    img = sitk.ReadImage(file_path)
    # 轉為 NumPy 陣列
    img_arr = sitk.GetArrayFromImage(img)

    return img_arr

def resize_volume(img):
    """Resize across z-axis"""
    # Set the desired depth
    desired_depth = 64
    desired_width = 128
    desired_height = 128
    
    # Get current depth
    current_width = img.shape[0]
    current_height = img.shape[1]
    current_depth = img.shape[-1]
    
    # Compute depth factor
    depth = current_depth / desired_depth
    width = current_width / desired_width
    height = current_height / desired_height
    depth_factor = 1 / depth
    width_factor = 1 / width
    height_factor = 1 / height
    # Rotate
    img = ndimage.rotate(img, 90, reshape=False)
    # Resize across z-axis
    img = ndimage.zoom(img, (width_factor, height_factor, depth_factor), order=1)
    return img

def preprocess_img(file_path):
    img = read_nifti_file(file_path)
    return resize_volume(img)

def show_slices(slices):
    fig, axes = plt.subplots(1, len(slices))
    for i, slice in enumerate(slices):
        axes[i].imshow(slice, cmap="gray", origin="lower")

In [ ]:
data_dir = 'C:\\Users\\Gina\\Lab\\kidney\\nnUNet-1\\Result'
mask_path = os.path.join(data_dir, 'case_00057.nii.gz')

data = read_nifti_file(mask_path)
resize_data = resize_volume(data)

In [ ]:
print('original image (black):', len(np.where(data == 0)[0]))
print('original image (kidney):', len(np.where(data == 1)[-1]))
print('original image (tumor):', np.where(data == 2))
print('-' * 20)
print('resize image (black):', len(np.where(resize_data == 0)[0]))
print('resize image (kidney):', len(np.where(resize_data == 1)[-1]))
print('resize image (tumor):', len(np.where(resize_data == 2)))

In [ ]:
def get_bounding_box(img):
    tumor_region = np.where(img == 2)

    if len(tumor_region[0]) == 0:
        return (666, 666, 666), (0, 0, 0)

    l_width = min(tumor_region[0])
    l_height = min(tumor_region[1])
    l_depth = min(tumor_region[2])

    r_width = max(tumor_region[0])
    r_height = max(tumor_region[1])
    r_depth = max(tumor_region[2])

    # turn the kidney part to black
    # img[np.where(img == 1)] = 0
    # img = img[l_width:r_width, l_height:r_height, l_depth:r_depth]

    return (l_width, l_height, l_depth), (r_width, r_height, r_depth)

def get_tumor_region(file_path):
    img = read_nifti_file(file_path)
    return get_bounding_box(img)


In [ ]:
# 顯示各軸切面
b_img = get_bounding_box(data)
print('image (tumor):', np.where(b_img == 2))

show_slices([b_img[:, :, 0], b_img[:, :, 1], b_img[:, :, 2]])
# show_slices([resize_data[:, :, 29], resize_data[:, :, 30], resize_data[:, :, 48]])

## Data For predicting tumor

In [ ]:
# import label data
with open('C:\\Users\\Gina\\Lab\\kidney\\kits21\\kits21\\data\\kits.json') as f:
    data = json.load(f)

label = [ case['tumor_histologic_subtype'] for case in data ]

In [ ]:
useless_type = ['rcc_unclassified',
                'urothelial',
                'mest',
                'collecting_duct_undefined',
                'oncocytoma',
                'clear_cell_papillary_rcc', 
                'multilocular_cystic_rcc', 
                'other', 'wilms', 
                'angiomyolipoma', 'spindle_cell_neoplasm']

drop_idx_img = []
for idx, value in enumerate(label):
    if value in useless_type:
        drop_idx_img.append(idx)

print('Useless img:', drop_idx_img)

In [ ]:
#  input mask
data_dir = 'C:\\Users\\Gina\\Lab\\kidney\\nnUNet-1\\Result'
mask_path = []

for file in os.listdir(data_dir):
    if "nii.gz" in file:
        # if int(file[5:10]) not in drop_idx_img:
        mask_path.append(os.path.join(data_dir, file))


In [ ]:
mask = []
l_width = 666
l_height = 666
l_depth = 666

r_width = 0
r_height = 0
r_depth = 0

bounding_box = []

for path in mask_path:
    img = read_nifti_file(path)

    # get the bounding box
    l_box, r_box = get_bounding_box(img)

    _box = (r_box[0] - l_box[0], r_box[1] - l_box[1], r_box[2] - l_box[2])
    bounding_box.append(_box)
    # print('bouding box:', _box)
    l_width = min(l_width, l_box[0])
    l_height = min(l_height, l_box[1])
    l_depth = min(l_depth, l_box[2])

    r_width = max(r_width, r_box[0])
    r_height = max(r_height, r_box[1])
    if r_box[2] > r_depth:
        print(r_box[2])
    r_depth = max(r_depth, r_box[2])

    mask.append(img)

# print(f"left: ({l_width}, {l_height}, {l_depth}), right: ({r_width}, {r_height}, {r_depth})")

mask = np.array(mask)
label = np.array(label)

In [ ]:
l_width = 666
l_height = 666
l_depth = 666

r_width = 0
r_height = 0
r_depth = 0

max_depth = 0

for path in mask_path:
    img = read_nifti_file(path)

    # get the bounding box
    l_box, r_box = get_bounding_box(img)

    _box = (r_box[0] - l_box[0], r_box[1] - l_box[1], r_box[2] - l_box[2])
    print('bouding box:', _box)
    
    if _box[2] > max_depth:
        max_depth = _box[2]
        print('max depth:', max_depth)
    # l_width = min(l_width, l_box[0])
    # l_height = min(l_height, l_box[1])
    # l_depth = min(l_depth, l_box[2])

    # r_width = max(r_width, r_box[0])
    # r_height = max(r_height, r_box[1])
    # if r_box[2] > r_depth:
    #     print(r_box[2])
    # r_depth = max(r_depth, r_box[2])

# print(f"left: ({l_width}, {l_height}, {l_depth}), right: ({r_width}, {r_height}, {r_depth})")

In [ ]:
print(np.where(mask[4] == 2)[2][:10])
show_slices([mask[4][:, :, 23 ], mask[4][:, :, 44], mask[4][:, :, 43]])

In [ ]:
print(f"left: ({l_width}, {l_height}, {l_depth}), right: ({r_width}, {r_height}, {r_depth})")

In [ ]:
# for idx, _ in enumerate(mask):
#     if _ is None:
#         drop_idx_img.append(idx)
#         print(idx, 'has no tumor')

# print('drop index:', drop_idx_img[-2:])
# mask = np.delete(mask, drop_idx_img[-2:])
# print(len(mask))

In [ ]:
label = np.delete(label, drop_idx_img)

In [ ]:
for _ in mask[:10]:
    print(_.shape)

In [ ]:
mask[0][42:503, 107:429].shape

In [ ]:
def resize_volume(img):
    """Resize across z-axis"""
    # left: (42, 107, 0), right: (503, 429, 911)
    img = img[42:503, 107:429]

    # Set the desired depth
    desired_depth = 200 // 2
    desired_width = 461 // 2
    desired_height = 322 // 2
    
    # Get current depth
    current_width = img.shape[0]
    current_height = img.shape[1]
    current_depth = img.shape[-1]

    # Compute depth factor
    depth = current_depth / desired_depth
    width = current_width / desired_width
    height = current_height / desired_height
    depth_factor = 1 / depth
    width_factor = 1 / width
    height_factor = 1 / height

    # turn the kidney part to black
    img[np.where(img == 1)] = 0

    # Rotate
    # img = ndimage.rotate(img, 90, reshape=False)
    # Resize across z-axis
    img = ndimage.zoom(img, (width_factor, height_factor, depth_factor), order=1)
    return img


In [ ]:
print('before resizing, the tumor position:')
print(np.where(mask[0] == 2)[2][:5])
print(np.where(mask[1] == 2)[2][:5])
print(np.where(mask[2] == 2)[2][:5])

In [ ]:
show_slices([mask[0][:, :, 319 ], mask[0][:, :, 320 ], mask[0][:, :, 321 ]])
show_slices([mask[1][:, :, 339 ], mask[1][:, :, 340 ], mask[1][:, :, 338 ]])
show_slices([mask[2][:, :, 177 ], mask[2][:, :, 175 ], mask[2][:, :, 176 ]])


In [ ]:
mask = np.array([ resize_volume(_mask) for _mask in mask ])

In [ ]:
print('after resizing, the tumor position:')
print(np.where(mask[0] == 2)[2][:5])
print(np.where(mask[1] == 2)[2][:5])
print(np.where(mask[2] == 2)[2][:5])

In [ ]:
show_slices([mask[0][:, :, 52 ], mask[0][:, :, 53 ], mask[0][:, :, 51 ]])
show_slices([mask[1][:, :, 56 ], mask[1][:, :, 55 ], mask[1][:, :, 57 ]])
show_slices([mask[2][:, :, 29 ], mask[2][:, :, 23 ], mask[2][:, :, 30 ]])

In [ ]:
def scipy_rotate(img_numpy):
    # define some rotation angles
    # angles = [-20, -10, -5, 5, 10, 20]
    # pick angles at random
    # angle = random.choice(angles)
    # rotate volume
    # volume = ndimage.rotate(volume, angle, reshape=False)
    # volume[volume < 0] = 0.0
    # volume[volume > 1] = 1.0
    # return volume
    """
    Returns a random rotated array in the same shape
    :param img_numpy: 3D numpy array
    :param min_angle: in degrees
    :param max_angle: in degrees
    """
    min_angle = -20
    max_angle = 20

    assert img_numpy.ndim == 3, "provide a 3d numpy array"
    assert min_angle < max_angle, "min should be less than max val"
    assert min_angle > -360 or max_angle < 360

    all_axes = [(1, 0), (1, 2), (0, 2)]
    angle = np.random.randint(low=min_angle, high=max_angle+1)
    axes_random_id = np.random.randint(low=0, high=len(all_axes))
    axes = all_axes[axes_random_id]

    return ndimage.rotate(img_numpy, angle, axes=axes)

rotated_img = mask[0]
rotated_img = scipy_rotate(rotated_img)

show_slices([rotated_img[:, :, 52], rotated_img[:, :, 53 ], rotated_img[:, :, 51 ]])

## Tumor type used only

In [ ]:
label_dummy = pd.get_dummies(label)
print(label_dummy)

## Grade used only

In [ ]:
label_grade = pd.DataFrame([ row['tumor_isup_grade'] for row in data ], columns=['grade'])

label_grade[ label_grade <= 2] = 0
label_grade[ label_grade > 2] = 1

In [ ]:
drop_index = label_grade[label_grade['grade'].isna()].index.tolist()

# drop nan grade index
print('before, label shape:', label_grade.shape)
print('before, mask shape:', mask.shape)

label_grade = label_grade.drop(index=drop_index)
mask = np.delete(mask, drop_index, axis=0)

print('after, label shape:', label_grade.shape)
print('after, mask shape:', mask.shape)

In [ ]:
print(np.where(mask[0] == 2)[2][:5])
print(np.where(mask[1] == 2)[2][:5])
print(np.where(mask[2] == 2)[2][:5])

In [ ]:
print('grade:', label_grade.loc[0])
show_slices([mask[0][:, :, 52], mask[0][:, :, 53], mask[0][:, :, 54]])

print('grade:', label_grade.loc[1])
show_slices([mask[1][:, :, 56], mask[1][:, :, 55], mask[1][:, :, 57]])

print('grade:', label_grade.loc[2])
show_slices([mask[2][:, :, 29], mask[2][:, :, 23], mask[2][:, :, 25]])

In [ ]:
from sklearn.model_selection  import train_test_split

x_train, x_val, y_train, y_val = train_test_split(mask, label_grade, test_size = 0.2)

print(
    "Number of samples in train and validation are %d and %d."
    % (x_train.shape[0], x_val.shape[0])
)

In [ ]:
print(y_train.value_counts(), '\n')
print(y_val.value_counts())

In [ ]:
print(np.where(x_train[0] == 2)[2][:5])
print(np.where(x_train[1] == 2)[2][:5])
print(np.where(x_train[2] == 2)[2][:5])

In [ ]:
show_slices([x_train[0][:, :, 30], x_train[0][:, :, 29], x_train[0][:, :, 31]])
show_slices([x_train[1][:, :, 49], x_train[1][:, :, 50], x_train[1][:, :, 51]])
show_slices([x_train[2][:, :, 70], x_train[2][:, :, 72], x_train[2][:, :, 71]])

In [ ]:
@tf.function
def rotate(volume):
    """Rotate the volume by a few degrees"""

    def scipy_rotate(img_numpy):
        # define some rotation angles
        # angles = [-20, -10, -5, 5, 10, 20]
        # pick angles at random
        # angle = random.choice(angles)
        # rotate volume
        # volume = ndimage.rotate(volume, angle, reshape=False)
        # volume[volume < 0] = 0.0
        # volume[volume > 1] = 1.0
        # return volume
        """
        Returns a random rotated array in the same shape
        :param img_numpy: 3D numpy array
        :param min_angle: in degrees
        :param max_angle: in degrees
        """
        min_angle = -20
        max_angle = 20

        assert img_numpy.ndim == 3, "provide a 3d numpy array"
        assert min_angle < max_angle, "min should be less than max val"
        assert min_angle > -360 or max_angle < 360

        all_axes = [(1, 0), (1, 2), (0, 2)]
        angle = np.random.randint(low=min_angle, high=max_angle+1)
        axes_random_id = np.random.randint(low=0, high=len(all_axes))
        axes = all_axes[axes_random_id]
        
        return ndimage.rotate(img_numpy, angle, axes=axes)

    augmented_volume = tf.numpy_function(scipy_rotate, [volume], tf.float16)
    return augmented_volume

def train_preprocessing(volume, label):
    """Process training data by rotating and adding a channel."""
    # Rotate volume
    volume = rotate(volume)
    volume = tf.expand_dims(volume, axis=3)
    # print(volume.get_shape())
    label = tf.cast(label, tf.float16)
    return volume, label

def validation_preprocessing(volume, label):
    """Process validation data by only adding a channel."""
    volume = tf.expand_dims(volume, axis=3)
    label = tf.cast(label, tf.float16)
    return volume, label


In [ ]:
# Define data loaders.
train_loader = tf.data.Dataset.from_tensor_slices((x_train, y_train.to_numpy()))
validation_loader = tf.data.Dataset.from_tensor_slices((x_val, y_val.to_numpy()))

batch_size = 3
# Augment the on the fly during training.
train_dataset = (
    train_loader.shuffle(len(x_train))
    .map(train_preprocessing)
    .batch(batch_size)
    .prefetch(2)
)
# Only rescale.
validation_dataset = (
    validation_loader.shuffle(len(x_val))
    .map(validation_preprocessing)
    .batch(batch_size)
    .prefetch(2)
)

In [ ]:
print(train_dataset.take(1))

In [ ]:
def get_model(width=128, height=128, depth=64):
    """Build a 3D convolutional neural network model."""

    inputs = keras.Input((width, height, depth, 1))

    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(inputs)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=128, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=256, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.GlobalAveragePooling3D()(x)
    x = layers.Dense(units=256, activation="relu")(x)
    x = layers.Dropout(0.3)(x)

    # outputs = layers.Dense(units=3, activation="softmax")(x) # type output
    outputs = layers.Dense(units=1, activation="sigmoid")(x) # grade output

    # Define the model.
    model = keras.Model(inputs, outputs, name="3dcnn")
    return model


# Build model.
model = get_model(width=230, height=161, depth=100)
model.summary()


In [ ]:
# Compile model.
initial_learning_rate = 0.0001
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
)
model.compile(
    loss="binary_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
    metrics=["acc", tf.keras.metrics.AUC()],
)

# Define callbacks.
checkpoint_cb = keras.callbacks.ModelCheckpoint(
    "3d_image_classification.h5", save_best_only=True
)
early_stopping_cb = keras.callbacks.EarlyStopping(monitor="val_acc", patience=15)

# Train the model, doing validation at the end of each epoch
epochs = 100
model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=epochs,
    shuffle=True,
    verbose=2,
    callbacks=[checkpoint_cb, early_stopping_cb],
)


In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 3))
ax = ax.ravel()

for i, metric in enumerate(["acc", "loss"]):
    # print(i, '. train:', model.history.history[metric])
    # print(i, '. val:', model.history.history["val_" + metric])
    ax[i].plot(model.history.history[metric])
    ax[i].plot(model.history.history["val_" + metric])
    ax[i].set_title("Model {}".format(metric))
    ax[i].set_xlabel("epochs")
    ax[i].set_ylabel(metric)
    ax[i].legend(["train", "val"])


In [ ]:
model.predict(train_dataset)

In [ ]:
def get_model_2(width=128, height=128, depth=64):
    """Build a 3D convolutional neural network model."""

    inputs = keras.Input((width, height, depth, 1))

    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(inputs)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=128, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.GlobalAveragePooling3D()(x)
    x = layers.Dense(units=128, activation="relu")(x)
    x = layers.Dropout(0.3)(x)

    # outputs = layers.Dense(units=3, activation="softmax")(x)
    outputs = layers.Dense(units=1, activation="sigmoid")(x)


    # Define the model.
    model = keras.Model(inputs, outputs, name="3dcnn")
    return model


# Build model.
model2 = get_model(width=230, height=161, depth=100)
model2.summary()


In [ ]:
# Compile model.
initial_learning_rate = 0.0001
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
)
# loss=tf.keras.losses.CategoricalCrossentropy(),
model2.compile(
    loss="binary_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
    metrics=["acc", tf.keras.metrics.AUC()],
)

# Define callbacks.
checkpoint_cb = keras.callbacks.ModelCheckpoint(
    "3d_image_classification_2.h5", save_best_only=True
)
early_stopping_cb = keras.callbacks.EarlyStopping(monitor="val_loss", patience=15)

# Train the model, doing validation at the end of each epoch
epochs = 100
model2.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=epochs,
    shuffle=True,
    verbose=2,
    callbacks=[checkpoint_cb, early_stopping_cb],
)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 3))
ax = ax.ravel()

for i, metric in enumerate(["acc", "loss"]):
    # print(i, '. train:', model.history.history[metric])
    # print(i, '. val:', model.history.history["val_" + metric])
    ax[i].plot(model2.history.history[metric])
    ax[i].plot(model2.history.history["val_" + metric])
    ax[i].set_title("Model {}".format(metric))
    ax[i].set_xlabel("epochs")
    ax[i].set_ylabel(metric)
    ax[i].legend(["train", "val"])

## 09/08
* 可以改成分 Low grade, High grade，應該比較好分
* 先把tumor這塊切出來，Cube 包起 tumor，只用這 cube，找最小的bounding box，再拉出來看